In [7]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep  9 14:13:37 2017

@author: stevewyl
"""
# 保证映射后结构一样
from keras.preprocessing.sequence import pad_sequences
# 文本预处理
from keras.preprocessing.text import Tokenizer
# 将类别映射成需要的格式
from keras.utils.np_utils import to_categorical

# 这个是连接层
from keras.layers.merge import concatenate

# 搭建模型
from keras.models import Sequential, Model

# 这个是层的搭建
from keras.layers import Dense, Embedding, Activation, Input

from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D

from keras.layers import  BatchNormalization
from keras.layers import Conv1D,MaxPooling1D

# 数据分割
from sklearn.model_selection import train_test_split

# 数据管道
from sklearn.pipeline import Pipeline,make_pipeline
# 数据处理
from data_helper_ml import load_data_and_labels

# 数据可视化
import matplotlib.pyplot as plt

good_data_file = "./data/good_cut_jieba.txt"
bad_data_file = "./data/bad_cut_jieba.txt"
mid_data_file = "./data/mid_cut_jieba.txt"
x_text, y = load_data_and_labels(good_data_file, bad_data_file, mid_data_file)

# Tokenizer是一个用于向量化文本，或将文本转换为序列
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tokenizer.fit_on_texts(x_text)
vocab = tokenizer.word_index

# 数据分割
x_train, x_test, y_train, y_test = train_test_split(x_text, y, test_size=0.2, random_state=2017)
#映射成数字
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
#让他共同化
x_train_padded_seqs = pad_sequences(x_train_word_ids, maxlen=64)
x_test_padded_seqs = pad_sequences(x_test_word_ids, maxlen=64)

In [4]:
print(len(x_text))
print(len(y))

15648
15648


# CNN模型

In [5]:
model = Sequential()
model.add(Embedding(len(vocab) + 1, 300, input_length=64)) #使用Embeeding层将每个词编码转换为词向量
model.add(Conv1D(256, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(128, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization())  # (批)规范化层
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 300)           4594500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 256)           384256    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 22, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 22, 128)           163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 64)             24640     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)              

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels,epochs=5, batch_size=800)
y_predict = model.predict_classes(x_test_padded_seqs)  # 预测的是类别，结果就是类别号
y_predict = list(map(str, y_predict))


C:\anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
  800/12518 [>.............................] - ETA: 1:05 - loss: 1.1991 - accuracy: 0.3688

In [12]:
from sklearn.metrics import accuracy_score,f1_score

In [15]:
# print('准确率', accuracy_score(y_test, y_predict))
# print('平均f1-score:',f1_score(y_test, y_predict, average='weighted'))

# textCNN

In [17]:
main_input = Input(shape=(64,), dtype='float64')
# 词嵌入（使用预训练的词向量）
embedder = Embedding(len(vocab) + 1, 300, input_length=64, trainable=False)

embed = embedder(main_input)

# 词窗大小分别为3,4,5
cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)

cnn1 = MaxPooling1D(pool_size=48)(cnn1)

cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)

cnn2 = MaxPooling1D(pool_size=47)(cnn2)

cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)

cnn3 = MaxPooling1D(pool_size=46)(cnn3)

# 合并三个模型的输出向量
cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)

flat = Flatten()(cnn)

drop = Dropout(0.2)(flat)

main_output = Dense(3, activation='softmax')(drop)

model = Model(inputs=main_input, outputs=main_output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 64, 300)      4594500     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 64, 256)      230656      embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 64, 256)      307456      embedding_4[0][0]                
____________________________________________________________________________________________

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)



Epoch 1/10
12518/12518 [==============================] - 47s 4ms/step - loss: 1.0583 - accuracy: 0.4372
Epoch 2/10
12518/12518 [==============================] - 48s 4ms/step - loss: 1.0242 - accuracy: 0.4754
Epoch 3/10
12518/12518 [==============================] - 50s 4ms/step - loss: 1.0010 - accuracy: 0.5000
Epoch 4/10
12518/12518 [==============================] - 52s 4ms/step - loss: 0.9741 - accuracy: 0.5218
Epoch 5/10
12518/12518 [==============================] - 48s 4ms/step - loss: 0.9466 - accuracy: 0.5429
Epoch 6/10
12518/12518 [==============================] - 50s 4ms/step - loss: 0.9223 - accuracy: 0.5582
Epoch 7/10
12518/12518 [==============================] - 50s 4ms/step - loss: 0.8992 - accuracy: 0.5678
Epoch 8/10
12518/12518 [==============================] - 48s 4ms/step - loss: 0.8768 - accuracy: 0.5848
Epoch 9/10
12518/12518 [==============================] - 47s 4ms/step - loss: 0.8555 - accuracy: 0.5971
Epoch 10/10
12518/12518 [==============================

In [21]:
import numpy as np
#y_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)  # 将标签转换为one-hot编码
result = model.predict(x_test_padded_seqs)  # 预测样本属于每个类别的概率

result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签

y_predict = list(map(str, result_labels))
# print(y_predict)
# print('准确率', metrics.accuracy_score(y_test, y_predict))

# print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

# 使用Word2Vec词向量的TextCNN

In [ ]:
w2v_model=Word2Vec.load('sentiment_analysis/w2v_model.pkl')
# 预训练的词向量中没有出现的词用0向量表示
embedding_matrix = np.zeros((len(vocab) + 1, 300))
for word, i in vocab.items():
    try:
        embedding_vector = w2v_model[str(word)]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue
        
 #构建TextCNN模型
def TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix):
    # 模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
    main_input = Input(shape=(50,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 300, input_length=50, weights=[embedding_matrix], trainable=False)
    #embedder = Embedding(len(vocab) + 1, 300, input_length=50, trainable=False)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=38)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=37)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=36)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(3, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
    one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=20)
    #y_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)  # 将标签转换为one-hot编码
    result = model.predict(x_test_padded_seqs)  # 预测样本属于每个类别的概率
    result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
    y_predict = list(map(str, result_labels))
    print('准确率', metrics.accuracy_score(y_test, y_predict))
    print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

In [3]:

# 导入使用到的库
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np


# One-Hot + MLP

In [27]:

# model = Sequential()
# # 全连接层
# model.add(Dense(512, input_shape=(len(vocab)+1,), activation='relu'))
# # DropOut层
# model.add(Dropout(0.5))
# # 全连接层+分类器
# model.add(Dense(3,activation='softmax'))
 
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# #  validation_data=(x_test, y_test)
# model.fit(x_train, y_train,
#           batch_size=32,
#           epochs=15
#           )

# RNN

In [31]:
# 模型结构：词嵌入-LSTM-全连接
model = Sequential()
model.add(Embedding(len(vocab)+1, 300, input_length=64))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.1))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 64, 300)           4594500   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 771       
Total params: 5,165,639
Trainable params: 5,165,639
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)

C:\anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
12518/12518 [==============================] - 80s 6ms/step - loss: 1.0250 - accuracy: 0.4867
Epoch 2/10
12518/12518 [==============================] - 74s 6ms/step - loss: 0.7618 - accuracy: 0.6507
Epoch 3/10
12518/12518 [==============================] - 75s 6ms/step - loss: 0.5988 - accuracy: 0.7440
Epoch 4/10
12518/12518 [==============================] - 74s 6ms/step - loss: 0.5064 - accuracy: 0.7956
Epoch 5/10
12518/12518 [==============================] - 78s 6ms/step - loss: 0.4221 - accuracy: 0.8364
Epoch 6/10
12518/12518 [==============================] - 74s 6ms/step - loss: 0.3631 - accuracy: 0.8647
Epoch 7/10
12518/12518 [==============================] - 74s 6ms/step - loss: 0.3184 - accuracy: 0.8824
Epoch 8/10
12518/12518 [==============================] - 76s 6ms/step - loss: 0.2824 - accuracy: 0.8958
Epoch 9/10
12518/12518 [==============================] - 74s 6ms/step - loss: 0.2551 - accuracy: 0.9065
Epoch 10/10
12518/12518 [==============================

In [33]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 64, 300)           4594500   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 771       
Total params: 5,165,639
Trainable params: 5,165,639
Non-trainable params: 0
_________________________________________________________________


# Bi-GRU

In [4]:
# 模型结构：词嵌入-双向GRU*2-全连接
model = Sequential()
# 64是序列号
model.add(Embedding(len(vocab)+1, 300, input_length=64))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1, return_sequences=True)))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1)))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 300)           4594500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64, 512)           855552    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1181184   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 6,632,775
Trainable params: 6,632,775
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)

C:\anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
 4800/12518 [==========>...................] - ETA: 5:04 - loss: 1.0801 - accuracy: 0.3985- ETA: 7:49 - loss: 1.0938 - accuracy: 

# CNN+RNN

C-LSTM串联（将CNN的输出直接拼接上RNN）

In [ ]:
# 模型结构：词嵌入-卷积池化-GRU*2-全连接
model = Sequential()
model.add(Embedding(len(vocab)+1, 300, input_length=64))
model.add(Convolution1D(256, 3, padding='same', strides = 1))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=2))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.1, return_sequences = True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.1))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)

# CNN+RNN并联

In [ ]:
# 模型结构：词嵌入-卷积池化-全连接 ---拼接-全连接
#                -双向GRU-全连接
main_input = Input(shape=(20,), dtype='float64')
embed = Embedding(len(vocab)+1, 300, input_length=64)(main_input)
cnn = Convolution1D(256, 3, padding='same', strides = 1, activation='relu')(embed)
cnn = MaxPool1D(pool_size=4)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(256)(cnn)
rnn = Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1))(embed)
rnn = Dense(256)(rnn)
con = concatenate([cnn,rnn], axis=-1)
main_output = Dense(3, activation='softmax')(con)
model = Model(inputs = main_input, outputs = main_output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)

# RCNN

In [6]:
# # 模型结构：词嵌入*3-LSTM*2-拼接-全连接-最大化池化-全连接
# # 我们需要重新整理数据集
# left_train_word_ids = [[len(vocab)] + x[:-1] for x in x_train_word_ids]
# left_test_word_ids = [[len(vocab)] + x[:-1] for x in x_test_word_ids]
# right_train_word_ids = [x[1:] + [len(vocab)] for x in x_train_word_ids]
# right_test_word_ids = [x[1:] + [len(vocab)] for x in x_test_word_ids]
 
# # 分别对左边和右边的词进行编码
# left_train_padded_seqs = pad_sequences(left_train_word_ids, maxlen=20)
# left_test_padded_seqs = pad_sequences(left_test_word_ids, maxlen=20)
# right_train_padded_seqs = pad_sequences(right_train_word_ids, maxlen=20)
# right_test_padded_seqs = pad_sequences(right_test_word_ids, maxlen=20)
 
# # 模型共有三个输入，分别是左词，右词和中心词
# document = Input(shape = (None, ), dtype = "int32")
# left_context = Input(shape = (None, ), dtype = "int32")
# right_context = Input(shape = (None, ), dtype = "int32")
 
# # 构建词向量
# embedder = Embedding(len(vocab) + 1, 300, input_length = 64)
# doc_embedding = embedder(document)
# l_embedding = embedder(left_context)
# r_embedding = embedder(right_context)
 
# # 分别对应文中的公式(1)-(7)
# forward = LSTM(256, return_sequences = True)(l_embedding) # 等式(1)
# # 等式(2)
# backward = LSTM(256, return_sequences = True, go_backwards = True)(r_embedding) 
# together = concatenate([forward, doc_embedding, backward], axis = 2) # 等式(3)
 
# semantic = TimeDistributed(Dense(128, activation = "tanh"))(together) # 等式(4)
# # 等式(5)
# pool_rnn = Lambda(lambda x: backend.max(x, axis = 1), output_shape = (128, ))(semantic) 
# output = Dense(3, activation = "softmax")(pool_rnn) # 等式(6)和(7)
# model = Model(inputs = [document, left_context, right_context], outputs = output)
# model.summary()
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
 
# model.fit([X_train_padded_seqs, left_train_padded_seqs, right_train_padded_seqs],y_train,
#            batch_size=32,
#            epochs=12,
#            validation_data=([X_test_padded_seqs, left_test_padded_seqs,right_test_padded_seqs], y_test))

# Attention

In [7]:
class Attention(Layer):
    def __init__(self, attention_size, **kwargs):
        self.attention_size = attention_size
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        # W: (EMBED_SIZE, ATTENTION_SIZE)
        # b: (ATTENTION_SIZE, 1)
        # u: (ATTENTION_SIZE, 1)
        self.W = self.add_weight(name="W_{:s}".format(self.name),
                                 shape=(input_shape[-1], self.attention_size),
                                 initializer="glorot_normal",
                                 trainable=True)
        self.b = self.add_weight(name="b_{:s}".format(self.name),
                                 shape=(input_shape[1], 1),
                                 initializer="zeros",
                                 trainable=True)
        self.u = self.add_weight(name="u_{:s}".format(self.name),
                                 shape=(self.attention_size, 1),
                                 initializer="glorot_normal",
                                 trainable=True)
        super(Attention, self).build(input_shape)
 
    def call(self, x, mask=None):
        # input: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        # et: (BATCH_SIZE, MAX_TIMESTEPS, ATTENTION_SIZE)
        et = K.tanh(K.dot(x, self.W) + self.b)
        # at: (BATCH_SIZE, MAX_TIMESTEPS)
        at = K.softmax(K.squeeze(K.dot(et, self.u), axis=-1))
        if mask is not None:
            at *= K.cast(mask, K.floatx())
        # ot: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        atx = K.expand_dims(at, axis=-1)
        ot = atx * x
        # output: (BATCH_SIZE, EMBED_SIZE)
        output = K.sum(ot, axis=1)
        return output
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [8]:
# 需要导入两个模型，分别是句子级别的和篇章级别的，以及预处理后的文本序列
def get_attention(sent_model, doc_model, sequences, topN=5):
    sent_before_att = K.function([sent_model.layers[0].input, K.learning_phase()],
                                 [sent_model.layers[2].output])
    cnt_reviews = sequences.shape[0]
 
    # 导出这个句子每个词的权重
    sent_att_w = sent_model.layers[3].get_weights()
    sent_all_att = []
    for i in range(cnt_reviews):
        sent_each_att = sent_before_att([sequences[i], 0])
        sent_each_att = cal_att_weights(sent_each_att, sent_att_w, model_name)
        sent_each_att = sent_each_att.ravel()
        sent_all_att.append(sent_each_att)
    sent_all_att = np.array(sent_all_att)
 
    doc_before_att = K.function([doc_model.layers[0].input, K.learning_phase()],
                                [doc_model.layers[2].output])
    # 找到重要的分句
    doc_att_w = doc_model.layers[3].get_weights()
    doc_sub_att = doc_before_att([sequences, 0])
    doc_att = cal_att_weights(doc_sub_att, doc_att_w, model_name)
 
    return sent_all_att, doc_att
 
# 使用numpy重新计算attention层的结果
def cal_att_weights(output, att_w, model_name):
    if model_name == 'HAN':
        eij = np.tanh(np.dot(output[0], att_w[0]) + att_w[1])
        eij = np.dot(eij, att_w[2])
        eij = eij.reshape((eij.shape[0], eij.shape[1]))
        ai = np.exp(eij)
        weights = ai / np.sum(ai)
        return weights

In [10]:
# 模型结构：词嵌入-双向GRU-Attention-全连接
inputs = Input(shape=(64,), dtype='float64')
embed = Embedding(len(vocab) + 1,300, input_length = 64)(inputs)
gru = Bidirectional(GRU(100, dropout=0.2, return_sequences=True))(embed)
attention = AttLayer()(gru)
output = Dense(3, activation='softmax')(attention)
model = Model(inputs, output)
model.summary()

NameError: name 'AttLayer' is not defined

# FastText

In [13]:
# 模型结构：词嵌入(n-gram)-最大化池化-全连接
# 生成n-gram组合的词(以3为例)
ngram = 3
# 将n-gram词加入到词表
def create_ngram(sent, ngram_value):
    return set(zip(*[sent[i:] for i in range(ngram_value)]))
ngram_set = set()
for sentence in x_train_padded_seqs:
    for i in range(2, ngram+1):
        set_of_ngram = create_ngram(sentence, i)
        ngram_set.update(set_of_ngram)
        
# 给n-gram词汇编码
start_index = len(vocab) + 2
token_indice = {v: k + start_index for k, v in enumerate(ngram_set)} # 给n-gram词汇编码
indice_token = {token_indice[k]: k for k in token_indice}
max_features = np.max(list(indice_token.keys())) + 1
# 将n-gram词加入到输入文本的末端
def add_ngram(sequences, token_indice, ngram_range):
    new_sequences = []
    for sent in sequences:
        new_list = sent[:]
        for i in range(len(new_list) - ngram_range + 1):
            for ngram_value in range(2, ngram_range + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)
    return new_sequences
  
x_train = add_ngram(x_train_word_ids, token_indice, ngram)
x_test = add_ngram(x_test_word_ids, token_indice, ngram)
x_train = pad_sequences(x_train, maxlen=25)
x_test = pad_sequences(x_test, maxlen=25)
 
model = Sequential()
model.add(Embedding(max_features, 300, input_length=64))
model.add(GlobalAveragePooling1D())
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 64, 300)           87423600  
_________________________________________________________________
global_average_pooling1d_1 ( (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 903       
Total params: 87,424,503
Trainable params: 87,424,503
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
one_hot_labels=y_train
model.fit(x_train_padded_seqs, one_hot_labels, batch_size=800, epochs=10)

C:\anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
12518/12518 [==============================] - 79s 6ms/step - loss: 1.0832 - accuracy: 0.4070A: 23s - loss: 1.0874 - accuracy:
Epoch 2/10
12518/12518 [==============================] - 66s 5ms/step - loss: 1.0559 - accuracy: 0.4493
Epoch 3/10
12518/12518 [==============================] - 61s 5ms/step - loss: 1.0265 - accuracy: 0.5006TA: 58s - loss: 1.0406 - a
Epoch 4/10
12518/12518 [==============================] - 64s 5ms/step - loss: 0.9956 - accuracy: 0.5141
Epoch 5/10
12518/12518 [==============================] - 77s 6ms/step - loss: 0.9620 - accuracy: 0.5701
Epoch 6/10
12518/12518 [==============================] - 66s 5ms/step - loss: 0.9252 - accuracy: 0.5915
Epoch 7/10
12518/12518 [==============================] - 76s 6ms/step - loss: 0.8844 - accuracy: 0.6394
Epoch 8/10
12518/12518 [==============================] - 82s 7ms/step - loss: 0.8420 - accuracy: 0.6683
Epoch 9/10
12518/12518 [==============================] - 94s 7ms/step - loss: 0.8012 - accuracy: 0.6